In [2]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

In [3]:
start_year = 2019
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s.PREC_ACC_NC.values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'p': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)
                        

2024-10-06 17:12:43
2024-10-06 17:26:22
2024-10-06 17:39:09
2024-10-06 17:51:50
2024-10-06 18:04:41
2024-10-06 18:16:43
2024-10-06 18:29:27
2024-10-06 18:41:45
2024-10-06 18:54:48
2024-10-06 19:07:08
2024-10-06 19:20:38
2024-10-06 19:34:25
2024-10-06 19:47:15
2024-10-06 20:00:40
2024-10-06 20:13:36
2024-10-06 20:26:56
2024-10-06 20:40:25
2024-10-06 20:52:26
2024-10-06 21:06:16
2024-10-06 21:20:18
2024-10-06 21:33:17
2024-10-06 21:46:13
2024-10-06 21:59:13
2024-10-06 22:12:09


In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 2019
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:43:14
2024-11-02 20:56:45
2024-11-02 21:10:09
2024-11-02 21:23:55
2024-11-02 21:39:04
2024-11-02 21:52:28
2024-11-02 22:06:15
2024-11-02 22:19:51
2024-11-02 22:33:47
2024-11-02 22:46:59
2024-11-02 23:01:51
2024-11-02 23:15:45
2024-11-02 23:29:37
2024-11-02 23:43:14
2024-11-02 23:56:33
2024-11-03 00:09:54
2024-11-03 00:23:06
2024-11-03 00:35:29
2024-11-03 00:50:27
2024-11-03 01:03:27
2024-11-03 01:17:02
2024-11-03 01:31:42
2024-11-03 01:45:33
2024-11-03 01:59:18


In [3]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 2019
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 01:12:44
2024-11-03 01:28:24
2024-11-03 01:43:08
2024-11-03 01:58:02
2024-11-03 02:13:31
2024-11-03 02:27:13
2024-11-03 02:40:49
2024-11-03 02:54:08
2024-11-03 03:08:40
2024-11-03 03:22:07
2024-11-03 03:36:37
2024-11-03 03:50:49
2024-11-03 04:04:45
2024-11-03 04:18:50
2024-11-03 04:32:10
2024-11-03 04:46:23
2024-11-03 05:00:42
2024-11-03 05:12:30
2024-11-03 05:26:23
2024-11-03 05:39:02
2024-11-03 05:52:16
2024-11-03 06:04:30
2024-11-03 06:17:20
2024-11-03 06:26:58


In [3]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 1995
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:10:37
2025-02-17 01:10:52
2025-02-17 01:11:15
2025-02-17 01:11:39
2025-02-17 01:11:59
2025-02-17 01:12:19
2025-02-17 01:12:38
2025-02-17 01:12:54
2025-02-17 01:13:09
2025-02-17 01:13:28
2025-02-17 01:13:48
2025-02-17 01:14:15
2025-02-17 01:14:31
2025-02-17 01:14:47
2025-02-17 01:15:03
2025-02-17 01:15:21
2025-02-17 01:15:40
2025-02-17 01:15:56
2025-02-17 01:16:16
2025-02-17 01:16:32
2025-02-17 01:16:48
2025-02-17 01:17:06
2025-02-17 01:17:22
2025-02-17 01:17:39
2025-02-17 01:17:56
2025-02-17 01:18:12
2025-02-17 01:18:27
2025-02-17 01:18:43
2025-02-17 01:19:06
2025-02-17 01:19:21
2025-02-17 01:19:38
2025-02-17 01:19:54
2025-02-17 01:19:56
2025-02-17 01:20:10
2025-02-17 01:20:29
2025-02-17 01:20:53
2025-02-17 01:21:10
2025-02-17 01:21:24
2025-02-17 01:21:40
2025-02-17 01:21:59
2025-02-17 01:22:16
2025-02-17 01:22:32
2025-02-17 01:22:52
2025-02-17 01:23:14
2025-02-17 01:23:34
2025-02-17 01:23:51
2025-02-17 01:24:11
2025-02-17 01:24:32
2025-02-17 01:24:53
2025-02-17 01:25:12
